# Embedding Model with Instructions

## Datasets

In [71]:
def display_dataset_info(dataset):
    info = dataset.info
    dataset_name = info.dataset_name
    splits_info = info.splits
    features = info.features
    print(f"Dataset Name: {dataset_name}")
    print("Splits Info:")
    for split_name, split_info in splits_info.items():
        num_examples = split_info.num_examples
        print(f" - Split: {split_name}, Num Examples: {num_examples}")
    print("Features:")
    for feature_name, feature_info in features.items():
        print(f" - {feature_name}: {feature_info}")

In [72]:
from datasets import load_dataset
import random
from collections import defaultdict

def get_dataset(dataset_name, train_size=0, test_size=0):  
    # Load the dataset
    dataset = load_dataset(dataset_name)
    
    # Access the train, test splits
    train_dataset = dataset['train']
    test_dataset = dataset['test']
    
    # Function to balance and shuffle a split
    def shuffle(split, size):
        if size == 0:
            return split
        label_to_indices = defaultdict(list)
        for idx, example in enumerate(split):
            label_to_indices[example['label']].append(idx)
        
        balanced_indices = []
        for indices in label_to_indices.values():
            if len(indices) >= size // len(label_to_indices):
                balanced_indices.extend(random.sample(indices, size // len(label_to_indices)))
            else:
                balanced_indices.extend(indices)
        
        random.shuffle(balanced_indices)
        return split.select(balanced_indices[:size])

    if(train_size != 0):
        train_dataset = train_dataset.shuffle(seed=42).select(range(train_size))
    if(test_size != 0):
        test_dataset = test_dataset.shuffle(seed=42).select(range(test_size))
    # Shuffle and balance the datasets
    # train_dataset = shuffle(train_dataset, train_size)
    # test_dataset = shuffle(test_dataset, test_size)
    
    return train_dataset, test_dataset

## Embedding Models

### BERT - Pipeline

In [73]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np

def encode_Pipeline(model, dataset, max_length=512, use_cls=True):
    key = "text"
    data = KeyDataset(dataset, key)
    pipe = model(data, return_tensors=True, truncation=True, padding=True, max_length=max_length)
    embeddings=[]
    for tensor in tqdm(pipe, desc="Encoding"): 
        # Tensor Shape [batch_size, sequence_length, hidden_size]
        if use_cls:
            embedding = tensor[:, 0, :]
        else:
            embedding = tensor.mean(dim=1)
        embeddings.append(embedding.squeeze())
    return np.array(embeddings), np.array(dataset["label"])

### Instructor - Sentence Transformers

In [74]:
def encode_ST(model, dataset):
    embeddings = []
    texts = dataset["text"]
    instructions = dataset["instruction"]
    for text, instruction in tqdm(zip(texts, instructions),total=len(dataset), desc="Encoding"):
        embedding = model.encode([[instruction, text]])
        embeddings.append(np.array(embedding).squeeze())
    return np.array(embeddings), np.array(dataset["label"])

### T5 - Transformer Sentence Piece

In [76]:
from transformers import T5Tokenizer, T5Model
import torch
from tqdm import tqdm
import numpy as np

def encode_T5(dataset, key="text", truncation=True, padding=True, max_length=512, use_mean_pooling=True):
    # Check if CUDA is available and set device accordingly
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize the tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-base")
    model = T5Model.from_pretrained("google-t5/t5-base").to(device)
    
    embeddings = []
    labels = []
    
    for data in tqdm(dataset, desc="Encoding text"):
        text = data[key]
        label = data.get("label", None)
        
        # Tokenize the input text
        inputs = tokenizer(text, truncation=truncation, padding=padding, max_length=max_length, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Get encoder outputs
        with torch.no_grad():
            encoder_outputs = model.encoder(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        
        # Pooling to get a single vector for each input
        if use_mean_pooling:
            attention_mask = inputs['attention_mask']
            last_hidden_state = encoder_outputs.last_hidden_state
            mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
            sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
            embedding = sum_embeddings / sum_mask
        else:
            embedding = encoder_outputs.last_hidden_state[:, 0, :]
        
        embeddings.append(embedding.cpu().numpy().flatten())  # Flatten the embeddings
        if label is not None:
            labels.append(label)
    
    return np.array(embeddings), np.array(labels)

## Augment Instruction

In [77]:
def mapper_affixes(example, prefix, suffix):
    example['text'] = prefix + example['text'] + suffix
    return example

def mapper_instruct(example, instruction):
    example['instruction'] = instruction
    return example

def augment_dataset_Affix(dataset, prefix, suffix):
    augmented_dataset = dataset.map(lambda x: mapper_affixes(x, prefix, suffix))
    return augmented_dataset

def augment_dataset_Inst(dataset, instruction):
    augmented_dataset = dataset.map(lambda x: mapper_instruct(x, instruction))
    return augmented_dataset

## Evaluation

In [78]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

def evaluate(method, train_embeddings, test_embeddings, train_labels, test_labels):
    if method == "SVM":
        model = SVC(kernel='linear')
        
    elif method == "MLP":
        model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                          solver='sgd', verbose=1, random_state=1,
                          learning_rate_init=.1)

    model.fit(train_embeddings, train_labels)
    predicted_labels = model.predict(test_embeddings)
    accuracy = accuracy_score(test_labels, predicted_labels)
    print("Report on " + method + ": ")
    print(classification_report(y_true = test_labels, y_pred = predicted_labels, digits=4))
    return round(accuracy,4)

## EmbedFlow

In [79]:
from transformers import pipeline
def EmbedFlow_Bert_1(train_dataset, test_dataset, evaluator, prefix, suffix=''):
    # Load Model
    model = pipeline("feature-extraction", model="google-bert/bert-base-uncased", device=0)
    
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Pipeline(model, train_dataset, max_length=512, use_cls=True)
    test_embeddings, test_labels = encode_Pipeline(model, test_dataset, use_cls=True)

    # Evaluate
    accuracy = evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)
    return accuracy

In [80]:
def EmbedFlow_Bert_2(train_dataset, test_dataset, evaluator, prefix, suffix=''):
    # Load Model
    model = pipeline("feature-extraction", model="google-bert/bert-large-uncased", device=0)
    
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Pipeline(model, train_dataset, max_length=512, use_cls=False)
    test_embeddings, test_labels = encode_Pipeline(model, test_dataset, max_length=512, use_cls=False)

    # Evaluate
    accuracy = evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)
    return accuracy

In [81]:
from sentence_transformers import SentenceTransformer
def EmbedFlow_Instructor(train_dataset, test_dataset, evaluator, instruction):
    # Load Model
    model = SentenceTransformer("hkunlp/instructor-large")
    
    # Add Instruction
    train_dataset = augment_dataset_Inst(train_dataset, instruction)
    test_dataset = augment_dataset_Inst(test_dataset, instruction)

    # Embed Dataset
    train_embeddings, train_labels = encode_ST(model, train_dataset)
    test_embeddings, test_labels = encode_ST(model, test_dataset)
    
    # Evaluate
    accuracy = evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)
    return accuracy

In [82]:
def EmbedFlow_T5(train_dataset, test_dataset, evaluator, prefix, suffix=''):
    # Load Model
    model = pipeline("feature-extraction", model="t5-base", device=0)

    # Add Instruction
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_T5(train_dataset)
    test_embeddings, test_labels = encode_T5(test_dataset)
    
    # Evaluate
    accuracy = evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)
    return accuracy

In [83]:
def EmbedFlow_GPT(train_dataset, test_dataset, evaluator, prefix, suffix=''):
    # Load Model
    model = pipeline("feature-extraction", model="openai-community/gpt2", device=0)
  
    # Add Instruction
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Pipeline(model, train_dataset, use_cls=False)
    test_embeddings, test_labels = encode_Pipeline(model, test_dataset, use_cls=False)
    
    # Evaluate
    accuracy = evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)
    return accuracy

## Testing
Logging: [Google Sheet](https://docs.google.com/spreadsheets/d/1iBDq7C59G6olf_of_sTF5oCY3Itj6_kImzeUl3XMpd8/edit#gid=1587051763)

In [84]:
import warnings
warnings.filterwarnings("ignore")
instructions_old = ['',
          'Movie Review: ', 
          'Restaurant Review: ', 
          'Sentiment Analysis: ', 
          'User Feedback: ', 
          'Customer Experience: ',
          'Product Review: ',
          'Service Feedback: ',
          'Experience at: ',
          'Abstract: ',
          'Research Paper Abstract: ',
          'Paper Summary: ']

instructions_general = [
    "Sentiment Analysis: ",
    "Summarize: ",
    "Review: ",
    "Evaluate: ",
    "Analyze: ",
    "Represent the text for determining the sentiment: ",
    "Summarize the main points of the text for a concise overview: ",
    "Evaluate the text to classify its sentiment and key themes: ",
    "Analyze the text for sentiment classification and thematic understanding: ",
    "Classify the sentiment and summarize the content of the given text: "
]

instructions_imdb=[
    "Movie Review: ",
    "Film Critique: ",
    "Cinema Opinion: ",
    "Film Feedback: ",
    "Review Analysis: ",
    "Represent the Review sentence for classifying emotion as positive or negative: ",
    "Evaluate the sentiment of the movie review sentence: ",
    "Analyze the review to determine if the sentiment is positive or negative: ",
    "Determine the emotional tone of the review for sentiment classification: ",
    "Classify the given movie review as either positive or negative: ",
]
instructions_yelp=[
    "Customer Experience at: ",
    "Opinion on: ",
    "Feed back on: ",
    "Restaurant Review: ",
    "User Feedback: ",
    "Describe the review in terms of customer satisfaction for the given place: ",
    "Evaluate the review to determine the customer's overall experience: ",
    "Analyze the review to classify the service quality as positive or negative: ",
    "Represent the customer's feedback on the establishment for sentiment analysis: ",
    "Determine the sentiment of the review, indicating whether the experience was positive or negative: ",
]

instructions_arXiv = [
    "Paper Summary: ",
    "Overview: ",
    "Abstract: ",
    "Key points: ",
    "Main findings: ",
    "Please read the following research paper abstract and summarize the key points and findings discussed: ",
    "Analyze the following abstract from a research paper and provide a detailed overview of its main ideas and conclusions: ",
    "Review the following abstract of a research paper and highlight the primary methodologies and results presented: ",
    "Examine the following abstract from a scientific paper and summarize the main topics and discoveries described: ",
    "Process the following research paper abstract and provide a concise summary of the core themes and contributions of the study: "
]

In [85]:
instructions_map = {
    'stanfordnlp/imdb': instructions_imdb,
    'yelp_review_full': instructions_yelp,
    'Voice49/arXiv-Abstract-Label-20k': instructions_arXiv
}

datasets = ['stanfordnlp/imdb', 
            'yelp_review_full',
            'Voice49/arXiv-Abstract-Label-20k']

evaluator = ['SVM', 'MLP']

train, test = get_dataset(datasets[0], train_size=1000, test_size=1000)

# Data structure to record accuracies
data = {
    "Instructions": ["None", "GS1", "GS2", "GS3", "GS4", "GS5", "GL1", "GL2", "GL3", "GL4", "GL5", 
                     "S1", "S2", "S3", "S4", "S5", "L1", "L2", "L3", "L4", "L5"],
    "Bert": [],
    "Bert-large": [],
    "Instructor": [],
    "T5": [],
    "GPT2": []
}

# Loop last dataset
for dataset in datasets[:1]:
    instructions = instructions_general + instructions_map[dataset]
    for instruction in instructions:
        print(f"Processing dataset: {dataset}, instruction: '{instruction}'")

        print("Bert")
        accB1 = EmbedFlow_Bert_1(train, test, evaluator[0], instruction)
        data["Bert"].append(accB1)

        print("Bert2")
        accB2 = EmbedFlow_Bert_2(train, test, evaluator[0], instruction)
        data["Bert-large"].append(accB2)

        print("Instructor")
        accI = EmbedFlow_Instructor(train, test, evaluator[0], instruction)
        data["Instructor"].append(accI)

        print("T5")
        accT = EmbedFlow_T5(train, test, evaluator[0], instruction)
        data["T5"].append(accT)

        print("GPT2")
        accG = EmbedFlow_GPT(train, test, evaluator[0], instruction)
        data["GPT2"].append(accG)

print(data)

Processing dataset: stanfordnlp/imdb, instruction: 'Sentiment Analysis: '
Bert


Encoding: 100%|██████████| 1000/1000 [00:07<00:00, 129.80it/s]


Report on SVM: 
              precision    recall  f1-score   support

           0     0.7960    0.7773    0.7866       512
           1     0.7720    0.7910    0.7814       488

    accuracy                         0.7840      1000
   macro avg     0.7840    0.7842    0.7840      1000
weighted avg     0.7843    0.7840    0.7840      1000

Bert2


Encoding:  72%|███████▏  | 717/1000 [00:14<00:05, 51.18it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Display the DataFrame
# print(df)

# Visualizing the data using appropriate plots
plt.figure(figsize=(14, 8))

for column in df.columns[1:]:
    plt.plot(df["Instructions"], df[column], marker='o', label=column)

plt.title("Classifier Accuracy on Text Embedding Models with Instructions")
plt.xlabel("Instructions")
plt.ylabel("Accuracy")
plt.legend(loc='upper center', bbox_to_anchor=(1.02, 1))
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate statistics for general, specific, and combined instructions
statistics = {}
for column in df.columns[1:]:
    general_instructions = df[column][1:11]  # GS1 to GL5
    specific_instructions = df[column][11:]  # S1 to L5
    combined_instructions = df[column][1:]   # All instructions (GS1 to L5)
    without_instructions = df[column][0]
    
    mean_general = general_instructions.mean()
    std_general = general_instructions.std()
    mean_specific = specific_instructions.mean()
    std_specific = specific_instructions.std()
    mean_combined = combined_instructions.mean()
    std_combined = combined_instructions.std()
    
    change_general = mean_general - without_instructions
    change_specific = mean_specific - without_instructions
    change_combined = mean_combined - without_instructions
    
    statistics[column] = {
        'Base': without_instructions,
        'Mean (G)': mean_general,
        'Mean (S)': mean_specific,
        'Mean (All)': mean_combined,
        'Std (G)': std_general,
        'Std (S)': std_specific,
        'Std (All)': std_combined,
        'Change (G)': change_general,
        'Change (S)': change_specific,
        'Change (All)': change_combined
    }

# Display the statistics
stats_df = pd.DataFrame(statistics).T
print(stats_df)